In [1]:
from config import config
config = config.Configuration()
run_prev_notebooks = False

In [2]:
import tensorflow as tf

import numpy as np
import rasterio
# from rasterio.windows import Window

import os
import time
import rasterio.warp # Reproject raster samples
from functools import reduce
from tensorflow.keras.models import load_model

from core.losses import tversky, accuracy, dice_coef, dice_loss, specificity, sensitivity
from core.optimizers import adaDelta #, adagrad, adam, nadam

import json
from sklearn.model_selection import train_test_split

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.patches import Polygon

import warnings # ignore annoying warnings
warnings.filterwarnings("ignore")
import logging
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)

%reload_ext autoreload
%autoreload 2
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Pipeline

In [3]:
def data_preparation():
    print("1: Data_preparation") 
    %run "1_data_preparation.ipynb"
def define_model():
    print("2: Model")
    %run "2_model.ipynb"
def training():
    print("3: Training")
    %run "3_training.ipynb"
def evaluation():
    print("4: Evaluation")
    %run "4_evaluation.ipynb"
def prediction():
    print("5: Prediction")
    %run "5_prediction.ipynb"
def data_cleanup():
    print("6: Data Cleanup")
    %run "6_data_cleanup.ipynb"

In [4]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ("data_preparation", data_preparation()),
    ("model", define_model()),
    ("training", training()),
    ("evaluation", evaluation()),
    # ("prediction", prediction()),
    # ("data_cleanup", data_cleanup())
])

1: Data_preparation
2: Model
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Input (InputLayer)             [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 256, 256, 64  1792        ['Input[0][0]']                  
                                )                                                                 
                                                                                                  
 conv2d_1 (Conv2D)              (None, 256, 256, 64  36928       ['conv2d[0][0]']                 
                                )                                

- Putting Everything Together (cf. p. 553 & davor)
    - Chaining Transformations
    - shuffle data etc.
- Präsi: Loading Preprocessing Data with TensorFlow